In [ ]:
from TTS.utils.manage import ModelManager

import sys
sys.path.insert(0, "../Src/")
import generateVoices as gv
from os.path import exists
import pandas as pd
from pydub import AudioSegment
import speech_recognition as sr
import jiwer
import json
import numpy as np

%autosave 5

In [ ]:
def evaluateWER(generatedAudioFileName, sentence):
    recognizer = sr.Recognizer()
    with sr.AudioFile(generatedAudioFileName) as source:
        audio = recognizer.record(source)
        hypothesis = recognizer.recognize_google(audio)

    wer = jiwer.wer(sentence, hypothesis)
    print("WER:", wer)
    return wer

In [ ]:
def evaluateWERForModel(model_dir_name, outputCSVFile):
    outputDF = pd.read_csv(outputCSVFile)
    speakers = outputDF['speakerId']
    wordErrorRateArray = []
    for speaker in speakers:
        sentence = outputDF[outputDF['speakerId'] == speaker]['generatedSentence']
        generatedAudioFileName = f'../Data/ttsOutputs/{model_dir_name}/{speaker}.wav'
        #evaluate word error rate for geneatedAudioFileName vs sentence
        wer = evaluateWER(generatedAudioFileName, sentence)
        wordErrorRateArray.append(wer)
    return np.mean(wordErrorRateArray)
    


In [ ]:
def main():
    wordErrorRateResults = []
    if exists('../Data/wordErrorRateResults.json'):
        with open('../Data/wordErrorRateResults.json', 'r') as f:
            wordErrorRateResults = json.load(f) 
    
    englishModels = gv.getEnglishModels()
    for model in englishModels:
        print(f"Processing model: {model}")
        model_dir_name = model.replace("/", "_")
        outputCSVFile = f'../Data/ttsOutputs/{model_dir_name}_generatedSentences.csv'
        if not exists(outputCSVFile):
            continue
        averageWER = evaluateWERForModel(model_dir_name, outputCSVFile)
        wordErrorRateResults.append({
            "model": model,
            "averageWER": averageWER
        })
        with open('../Data/wordErrorRateResults.json', 'w') as f:
            json.dump(wordErrorRateResults, f)
    wordErrorRateResults = pd.DataFrame(wordErrorRateResults)
    wordErrorRateResults = wordErrorRateResults.sort_values(by=['averageWER'], ascending=False)
    display(wordErrorRateResults)
    wordErrorRateResults.to_csv('../Data/wordErrorRateResults.csv', index=False)
    print('done')

In [ ]:
if __name__ == '__main__':
    main()